In [7]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from googletrans import Translator
from scipy.spatial.distance import cosine
from tqdm import tqdm
import compress_fasttext
%matplotlib inline
%pylab inline



Populating the interactive namespace from numpy and matplotlib


In [8]:


responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:53, 861.25it/s] 


In [9]:
train_list = []
for resp in responses:
#     print(list(resp.values()))
    train_list.append(list(resp.values()))
train = pd.DataFrame(columns = list(responses[99].keys()), data = train_list)
del train_list

In [10]:
train.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


In [11]:
# train[train['rating_grade'] == ]
train['rating_grade'].unique()

array([nan,  2.,  1.,  5.,  3.,  4.])

In [12]:
train[train['rating_grade'] == 5]

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
98,г. Санкт-Петербург,False,"Хорошо, когда проблемы решаются",0,лицензия № 1073,apka45,Русславбанк,2015-06-03 17:40:38,Досрочно закрывали вклады перед отпуском. Жена...,5.0
122,г. Нижний Новгород,False,Изменяют проценты по уже открытому договору вк...,3,лицензия № 226,20157673,РосинтерБанк,2015-06-04 09:38:18,"Хотел открыть вклад ""Доходный+"" под 19% год , ...",5.0
157,г. Москва,False,"Отделение Марксистская, дом 5: спасибо за помощь!",0,лицензия № 1623,ospar,ВТБ 24,2015-06-04 22:58:38,"Вчера, 3 июня, просидев в очереди более 3-х ча...",5.0
259,г. Москва,False,Хорошее обслуживание,0,лицензия № 3206,ser-gyr,Соверен Банк,2015-06-02 18:23:41,Сегодня открывал вклад в этом банке. Очередь б...,5.0
266,г. Москва,False,Моя история получения кредита в Ренессанс Кред...,1,лицензия № 3354,SkaIa,Ренессанс Кредит,2015-06-03 02:03:28,Моя история получения кредита в Ренессанс Кред...,5.0
...,...,...,...,...,...,...,...,...,...,...
153488,None,False,Повысилось качество обслуживания,0,лицензия № 1623,Атмос,ВТБ 24,2005-04-13 12:00:00,Пользуюсь банком 3 месяца - специально открыл ...,5.0
153489,None,False,Стабильный банк с высокими процентами,0,лицензия № 1971,Александра,Ханты-Мансийский банк Открытие,2005-04-14 13:41:00,А что писать? Ставки по вкладам высокие (до 13...,5.0
153491,None,False,Ну тогда копите денюжки и покупайте не в креди...,4,лицензия № 2289,irzik,Русский Стандарт,2005-04-14 17:42:00,А я с мужем обставила техникой совершенно нову...,5.0
153494,None,False,Не все так страшно,0,лицензия № 2557,Вера,Ситибанк,2005-05-18 14:38:00,"Слишком большой банк, не всегда учитывает нашу...",5.0


## В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

In [33]:
import re
re_del_spec = re.compile( r"[%&()*\"#/@;:<>{}`+=~|!?,.]+")

stop_words_russian = ["и", "в", "во",  "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", 
                      "его", "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", 
                      "вот", "от", "меня", "еще", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", 
                      "ли", "если", "уже", "или", "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", 
                      "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", 
                      "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", 
                      "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", 
                      "совсем", "ним", "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", 
                      "куда", "зачем", "всех", "никогда", "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", 
                      "над", "больше", "тот", "через", "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", 
                      "эту", "моя", "впрочем", "хорошо", "свою", "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", 
                      "такой", "им", "более", "всегда", "конечно", "всю", "между", "при", "однако", "это", "всё", "весь",
                      "обо", "ваш" , " " , "  ", " - ", "-", "   "]





tokens_5_star = []    
train_5_star = train[train['rating_grade'] == 5]
for index, row in tqdm(train_5_star.iterrows(), total = len(train)):
    tokens = []
    sample = [re_del_spec.sub(" ", word.lower()) for word in row['text'].split()]
    tokens_stop = [word for word in sample if word not in  stop_words_russian]
#     print(tokens_stop)
#     for word in tokens_stop:
#         tokens += sample.text.iloc[i].split()
    tokens_5_star.append(tokens_stop)



 10%|▉         | 14713/153499 [00:14<02:13, 1038.60it/s]


In [51]:

tokens_1_star = []    
train_1_star = train[train['rating_grade'] == 1]
for index, row in tqdm(train_1_star.iterrows(), total = len(train)):
    tokens = []
    sample = [re_del_spec.sub(" ", word.lower()) for word in row['text'].split()]
    tokens_stop = [word for word in sample if word not in  stop_words_russian]
#     print(tokens_stop)
#     for word in tokens_stop:
    tokens_1_star.append(tokens_stop)



 31%|███       | 47387/153499 [01:10<02:38, 667.68it/s] 


In [49]:


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
top_n = 40


tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0)
tfidf_matrix =  tfidf.fit_transform([' '.join(tokens) for tokens in tokens_5_star])
feature_names = tfidf.get_feature_names() 
indices = np.argsort(tfidf.idf_)[::-1]

top_features = [feature_names[i] for i in indices[:top_n]]
print(f"Топ {top_n } ключевых слов и биграмм для отзывов с рангом 5")
for feature in top_features:
    print(feature)

Топ 40 ключевых слов и биграмм для отзывов с рангом 5
ёё получению
кстати порядок
кстати последнее
кстати последней
кстати последних
кстати последнюю
кстати посмотрел
кстати посмотрите
кстати посоветовал
кстати постоянно
кстати поступившие
кстати поступить
кстати потратил
кстати почта
кстати пошел
кстати пошла
кстати появился
кстати посидеть
кстати порталом
кстати ответ
кстати пораньше
кстати получении
кстати получив
кстати получили
кстати получить
кстати пользовалась
кстати пользуюсь
кстати помимо
кстати поминутно
кстати понравилось
кстати пообещали
кстати пополнение
кстати попросил
кстати попросила
кстати попутно
кстати поражён
кстати права
кстати правда
кстати правильно
кстати праздниках


In [52]:

top_n = 40


tfidf_1 = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0)
tfidf_matrix_1 =  tfidf_1.fit_transform([' '.join(tokens) for tokens in tokens_1_star])
feature_names_1 = tfidf_1.get_feature_names() 
indices_1 = np.argsort(tfidf_1.idf_)[::-1]

top_features_1 = [feature_names_1[i] for i in indices_1[:top_n]]
print(f"Топ {top_n } ключевых слов и биграмм для отзывов с рангом 1")
for feature in top_features_1:
    print(feature)

Топ 40 ключевых слов и биграмм для отзывов с рангом 1
ёё стороны
несанкционированную смс
несанкционированные снятия
несанкционированные операции
несанкционированные начисления
волнуюсь вокруг
волнуюсь деньги
волнуюсь заявка
несанкционированные владельцем
волнуюсь звоню
несанкционированную транзакцию
несанкционированную рекламу
волнуйтеся бежите
несанкционированную рассылку
волнуюсь количества
волнуюсь мои
несанкционированному перерасходу
несанкционированному овердрафту
волнуюсь него
волнуюсь оплаты
волнуюсь пишу
несанкционированном разглашении
волнуюсь банкомат
волнуйтеся
волнуюсь получатель
несанкционированным списанием
волнуйтесь третьего
несанкционированных дистанционных
несанкционированных действиях
несанкционированных выпусков
волнуйтесь увидите
несанкционированными операциями
несанкционированными
несанкционированным транзакциям
несанкционированным также
несанкционированным снятием
несанкционированный вход
несанкционированным овердрафтом
несанкционированным изъятием
несанкциониров